In [ ]:
from stable_baselines3 import DQN
from emg_env_bu import emg_env
from cnn_feature_extractor import CNN_feature

if __name__ =='__main__':
    # 1. 데이터 경로 설정
    TRAIN_DATA_DIR = 'リアルタイム/spec_tensor/'
    TEST_DATA_DIR = 'data/test/' # 필요 시 테스트 데이터 경로도 설정
    # 2. 학습용 환경 생성
    env = emg_env(data_dir=TRAIN_DATA_DIR, train=True)
    # 3. 사용자 정의 CNN을 위한 policy_kwargs 설정
    policy_kwargs = dict(
        features_extractor_class = CNN_feature,
        features_extractor_kwargs = dict(features_dim=256),   #최종 특징 벡터 크기
    )
    
 # 4. DQN 모델 생성
    model = DQN(
        'CnnPolicy',
        env,
        policy_kwargs=policy_kwargs,
        buffer_size=50000,       # 리플레이 버퍼 크기
        learning_starts=1000,    # 학습 시작 전 최소 경험 수
        batch_size=64,           # 미니배치 크기
        learning_rate=1e-4,      # 학습률
        gamma=0.99,              # 할인 계수
        tau=1.0,                 # 타겟 네트워크 업데이트 강도
        train_freq=4,            # 훈련 빈도
        gradient_steps=1,        # 그래디언트 업데이트 스텝
        verbose=1,
        tensorboard_log="./logs/dqn_emg_custom/"
    )
    
    # 5. 모델 학습 시작
    print("===== 学習開始 =====")
    model.learn(total_timesteps=42000, log_interval=4)
    model.save("dqn_emg_final_model")
    print("===== 学習完了およびモデル保存 =====")
    
    env.close()
    

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
===== 学習開始 =====
Logging to ./logs/dqn_emg_custom/DQN_6
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 113      |
|    ep_rew_mean      | -75.5    |
|    exploration_rate | 0.957    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 287      |
|    time_elapsed     | 1        |
|    total_timesteps  | 452      |
----------------------------------
모든 데이터를 한 번씩 사용했습니다. 처음부터 다시 시작합니다.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 113      |
|    ep_rew_mean      | -76      |
|    exploration_rate | 0.914    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 293      |
|    time_elapsed     | 3        |
|    total_timesteps  | 904      |
----------------------------------
----------------------------------
| rollout

ValueError: 'data/test/' 경로에 ep_*.npz로 시작하는 NPZ 파일이 없습니다.

In [ ]:
    # (필요 시) 학습된 모델 테스트
test_env = emg_env(data_dir=TEST_DATA_DIR, train=False)
obs, info = test_env.reset()
done = False
while not done:
        action, _ = model.predict(obs, deterministic=True) 
        obs, reward, terminated, truncated, info = test_env.step(action)
        done = terminated or truncated
        print(f"예측: {action}, 실제: {test_env.episode_label}, 보상: {reward}")
        test_env.close()